In [15]:
from langchain_groq import ChatGroq

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
llm = ChatGroq(model="llama-3.3-70b-versatile")
res = llm.invoke("5 synonyms for the word power")

In [18]:
print(res.content)

Here are 5 synonyms for the word "power":

1. Strength
2. Authority
3. Control
4. Influence
5. Might


In [19]:
from langchain_ollama import ChatOllama

In [20]:
llm = ChatOllama(model="llama3.2:1b", temperature=1.8, num_predict=15)
res = llm.invoke("The first person to go on moon was")

In [21]:
print(res.content)

The first person to walk on the Moon was Neil Armstrong. He stepped onto


In [22]:
from langchain_core.prompts import PromptTemplate

In [23]:
pt = PromptTemplate.from_template("Give a couplet for {topic1} and {topic2}")

In [24]:
#pt = PromptTemplate.from_template("What is the difference between {t1}, {t2}, {t3}")

In [25]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [26]:
chain = pt | llm

In [ ]:
#response = chain.invoke({'t1':'Hyphen', 't2':'En Dash', 't3':'Em Dash'})
#print(response.content)

The hyphen (-), en dash (–), and em dash (—) are three types of dashes used in punctuation, each with its own specific uses and meanings.

1. **Hyphen (-)**: A hyphen is used to join two or more words together to form a compound word. It can also be used to indicate a range or connection between two things. Examples:
   - Self-made
   - Merry-go-round
   - Co-pilot

2. **En Dash (–)**: An en dash is used to indicate a range or connection between two things, such as dates, times, or numbers. It can also be used to replace "to" or "and" in certain contexts. Examples:
   - Monday–Friday (instead of "Monday to Friday")
   - 1980–1990 (instead of "1980 to 1990")
   - The New York–London flight (instead of "The New York to London flight")

3. **Em Dash (—)**: An em dash is used to indicate a pause or a break in thought. It can be used to set off a parenthetical remark, to indicate a sudden change in thought, or to add emphasis to a sentence. Examples:
   - I was going to the store—but then I

In [28]:
response = chain.invoke({'topic1':'Modi', 'topic2':'Politics'})
print(response.content)

Here's a couplet for Modi and Politics:

"Modi's charm on voters does shine,
In politics, his mark will forever be divine."


# JSON Output Parser

In [29]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [37]:
prompt ='''
From the below article, extract movie name, budget, revenue, studio name in JSON output format containing the following keys: 'movie_title', 'Budget', 'Revenue', 'Studio'

Add units like Million or Billion
e.g - $ 120 Million
Only return the valid JSON . No preamble

Artilce
=========
{article}


'''

In [38]:
pt = PromptTemplate.from_template(prompt)

In [39]:
article_text = '''
A new sci-fi blockbuster titled Stellar Horizons hit theaters last week, marking one of the most ambitious releases of the year. Produced by Nebula Pictures, the film had a massive production budget of $180 million, largely driven by cutting-edge CGI and on-location shoots across three continents.

Early box-office tracking shows the movie performing exceptionally well, bringing in a worldwide revenue of $620 million within the first two weeks. Critics praised the film’s world-building and visual style, and industry analysts predict the earnings will continue rising throughout the month.

'''

In [40]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [41]:
chain = pt | llm

In [42]:
response = chain.invoke({'article':article_text})
print(response.content)

```
{
  "movie_title": "Stellar Horizons",
  "Budget": "$180 Million",
  "Revenue": "$620 Million",
  "Studio": "Nebula Pictures"
}
```


In [44]:
type(response.content)

str

In [45]:
parser = JsonOutputParser()
res = parser.parse(response.content)
res

{'movie_title': 'Stellar Horizons',
 'Budget': '$180 Million',
 'Revenue': '$620 Million',
 'Studio': 'Nebula Pictures'}

In [46]:
res['Budget']

'$180 Million'

# Embeddings

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings  

In [2]:
embedder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\REVATI\AppData\Local\Temp\ipykernel_15748\2386877202.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\REVATI\Desktop\gen-ai\genv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text = "Data analytics helps companies optimize decision-making"

In [4]:
vector = embedder.embed_query(text)

In [5]:
vector[:10]

[0.01106224674731493,
 0.04219743236899376,
 -0.06616464257240295,
 0.023094290867447853,
 0.014837641268968582,
 -0.03678319975733757,
 0.07881171256303787,
 -0.05508483201265335,
 -0.019374774768948555,
 0.03352861851453781]

In [6]:
len(vector)

384

# Compare Two Sentences

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#text1 = "Machine Learning improves prediction accuracy"
#text2 = "AI models help make better predictions"
#text3 = "Pizza is my favorite food"
#text4 = "I am hungry, please prepare something"

In [13]:
text1 = "The sun rises in the east"
text2 = "Computers process data quickly"
text3 = "I like mangoes"
text4 = "Please open the window"

In [14]:
v1 = embedder.embed_query(text1)
v2 = embedder.embed_query(text2)
v3 = embedder.embed_query(text3)
v4 = embedder.embed_query(text4)

In [15]:
print("Similarity text1 <-> text2", cosine_similarity([v1], [v2]))
print("Similarity text1 <-> text3", cosine_similarity([v1], [v3]))
print("Similarity text3 <-> text2", cosine_similarity([v3], [v2]))
print("Similarity text3 <-> text4", cosine_similarity([v3], [v4]))

Similarity text1 <-> text2 [[0.02331105]]
Similarity text1 <-> text3 [[0.12525778]]
Similarity text3 <-> text2 [[0.09533108]]
Similarity text3 <-> text4 [[0.13273711]]


Imagine you are working as a data analyst in a bank.
Customers write Free-text complaints, and you want to automatically figure out which category a new complaint belongs to.

You'll use sentence transformers to convert text into embeddings.
Then use cosine similarity to match the new complaint to the closest category

Complaint Categories
1. Credit Card issues
2. Loan processing delay
3. Online banking login errors

New Customer Complaint(Query)
"I cannot access my bank account online.
It keeps showing an authentication error."

In [32]:
from sentence_transformers import SentenceTransformer

In [33]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [34]:
categories = [
      "Credit Card issues",
      "Loan processing delay",
      "Online banking login errors"
]

In [50]:
query = "My credit card payment was declined even though there is sufficient balance."

In [51]:
category_embeddings = model.encode(categories)
query_embedding = model.encode([query])

In [52]:
similarities = cosine_similarity(query_embedding, category_embeddings)

In [53]:
similarities

array([[0.5080988 , 0.31490117, 0.22870821]], dtype=float32)

In [54]:
import numpy as np

In [55]:
best_match_index = np.argmax(similarities)
print(f"\n Query:{query} \n Most likely category: ", categories[best_match_index])


 Query:My credit card payment was declined even though there is sufficient balance. 
 Most likely category:  Credit Card issues
